In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
import cv2
from keras.optimizers import Adam, SGD
from keras.models import load_model
import imutils

Using TensorFlow backend.


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
x = np.load('./drive/My Drive/lict/numta_x.npy')/255.0
y = np.load('./drive/My Drive/lict/numta_y.npy')

In [ ]:
x = x.reshape(-1,64,64,1)
from keras.utils import to_categorical
y = to_categorical(y)

In [ ]:
x.shape

(55267, 64, 64, 1)

In [ ]:
from keras import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dropout, Dense,ZeroPadding2D

In [ ]:
model = Sequential()
model.add(Conv2D(input_shape=(64,64,1),filters=64,kernel_size=(2,2),padding="same", activation="relu"))


model.add(Conv2D(filters=64,kernel_size=(2,2),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(2,2), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(2,2), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(2,2), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(2,2), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(2,2), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))


model.add(Flatten())

model.add(Dense(units=64,activation="relu"))
model.add(Dense(units=10, activation="softmax"))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y,shuffle=True, test_size=0.01)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rotation_range=5, width_shift_range=0.2, height_shift_range=0.2,  zoom_range=0.2)

datagen.fit(x_train)

In [ ]:
history= model.fit(datagen.flow(x_train, y_train,batch_size=250),validation_data=(x_test, y_test),epochs=20)

Epoch 1/20
219/219 [==============================] - 56s 257ms/step - loss: 2.3030 - accuracy: 0.0988 - val_loss: 2.3024 - val_accuracy: 0.1230
Epoch 2/20
219/219 [==============================] - 55s 252ms/step - loss: 2.3026 - accuracy: 0.0998 - val_loss: 2.3026 - val_accuracy: 0.0904
Epoch 3/20
219/219 [==============================] - 55s 252ms/step - loss: 2.3026 - accuracy: 0.1003 - val_loss: 2.3026 - val_accuracy: 0.0904
Epoch 4/20
219/219 [==============================] - 55s 251ms/step - loss: 2.3026 - accuracy: 0.0985 - val_loss: 2.3026 - val_accuracy: 0.1085
Epoch 5/20
219/219 [==============================] - 55s 251ms/step - loss: 2.3026 - accuracy: 0.0983 - val_loss: 2.3027 - val_accuracy: 0.0904
Epoch 6/20
219/219 [==============================] - 55s 251ms/step - loss: 2.3026 - accuracy: 0.0991 - val_loss: 2.3026 - val_accuracy: 0.0904
Epoch 7/20
219/219 [==============================] - 55s 251ms/step - loss: 2.3026 - accuracy: 0.0972 - val_loss: 2.3027 - val_ac

In [ ]:
model.save('weight.h5')

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model acc')
plt.ylabel('acc')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
def segment(image, height=64):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)
 
    # threshold the image
    ret,thresh1 = cv2.threshold(gray ,100,255,cv2.THRESH_BINARY_INV)
 
    # dilate the white portions
    dilate = cv2.dilate(thresh1, None, iterations=2)
 
    # find contours in the image
    cnts = cv2.findContours(dilate.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[1] if imutils.is_cv2() else cnts[0]
 
    orig = image.copy()
    i = 0
    t = 0
    c=0
    x_all = []
    w_all = []
    lines = []
    parts = []
 
    for cnt in cnts:
        # Check the area of contour, if it is very small ignore it
        if(cv2.contourArea(cnt) < 100):
            continue
 
        # Filtered countours are detected
        x,y,w,h = cv2.boundingRect(cnt)
        x_all.append(x)
        w_all.append(w)
        i = i + 1
 
    comb = np.zeros((2,len(x_all)))
    comb[0,:]=x_all
    comb[1,:]=w_all
    comb = comb.T
    comb = comb[comb[:,0].argsort()]
 
    x_all = comb[:,0]
    w_all = comb[:,1]
 
 
    for i, item in enumerate (x_all):
        if i < len(x_all)-1:
            lines.append((item+w_all[i]+x_all[i+1])/2)
 
 
    for i in range(len(lines)):
        parts.append(cv2.resize(image[:,t:int(lines[i]),:],(height,height)))
        t = int(lines[i])
        if i == len(lines)-1:
            parts.append(cv2.resize(image[:,t:,:],(height,height)))
   
    return parts

In [ ]:
img=cv2.imread('test1.jpg')

a=segment(img)
L=np.array(a)
L.shape

In [ ]:

result=[]
s=""
for i in range (0,4):
    img1=(cv2.resize(L[i],(64,64),0))
    p=np.argmax(model.predict(np.reshape(img1,(-1,64,64,1))))
    s=s+str(p)
   
plt.imshow(img)
plt.title(s)